In [ ]:
from datasets import load_dataset
ds = load_dataset("Synanthropic/reading-analog-gauge")

## Unoptimized for mac m1 GPU

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt

class GaugeReader:
    def __init__(self):
        self.input_shape = (224, 224, 3)
        self.min_value = 0
        self.max_value = 10
        
    def build_model(self):
        # Modified model architecture to predict single value
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1)  # Single output for gauge reading
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        # Load dataset from HuggingFace
        print("Loading dataset...")
        dataset = load_dataset("Synanthropic/reading-analog-gauge")
        train_ds = dataset['train']
        
        def preprocess_single_example(example):
            # Convert image to numpy array
            image = np.array(example['image'])
            
            # Resize image
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            
            # Normalize image
            image = image.astype(np.float32) / 255.0
            
            # Get label
            label = np.array(example['label'], dtype=np.float32)
            
            return {
                'image': image,
                'label': label
            }
        
        print("Processing examples...")
        # Process all examples
        processed_data = [preprocess_single_example(example) for example in train_ds]
        
        # Separate images and labels
        images = np.array([example['image'] for example in processed_data])
        labels = np.array([example['label'] for example in processed_data])
        
        print(f"Dataset shape - Images: {images.shape}, Labels: {labels.shape}")
        
        # Create TensorFlow dataset
        dataset = tf.data.Dataset.from_tensor_slices((images, labels))
        
        # Batch and shuffle
        dataset = dataset.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
        
        return dataset

    def train_model(self, epochs=10):
        # Build model
        model = self.build_model()
        
        print("Preparing dataset...")
        train_ds = self.prepare_dataset()
        print("Dataset prepared successfully!")
        
        # Train model
        print("Starting training...")
        history = model.fit(
            train_ds,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='loss',
                    patience=3,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    factor=0.5,
                    patience=2
                )
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            pickle.dump(tflite_model, f)
    
    def predict_value(self, model, image_path):
        # Load and preprocess image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
        image = image.astype(np.float32) / 255.0
        
        # Add batch dimension
        image = np.expand_dims(image, 0)
        
        # Predict
        prediction = model.predict(image)
        return float(prediction[0][0])

    def visualize_prediction(self, image_path, prediction):
        # Load and resize image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
        
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f'Predicted Value: {prediction:.2f}')
        plt.axis('off')
        plt.show()

def main():
    # Initialize GaugeReader
    gauge_reader = GaugeReader()
    
    # Train model
    print("Training model...")
    model, history = gauge_reader.train_model(epochs=10)
    
    # Convert to TFLite
    print("Converting to TFLite...")
    tflite_model = gauge_reader.convert_to_tflite(model)
    
    # Save model
    print("Saving model...")
    gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
    
    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='MAE')
    plt.title('Model MAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
# Check dataset structure
dataset = load_dataset("Synanthropic/reading-analog-gauge")
train_ds = dataset['train']
example = train_ds[0]
print("Dataset keys:", example.keys())

## Optimized for mac m1 GPU

In [2]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt
import gc  # Garbage collector
import os

class GaugeReader:
    def __init__(self):
        self.input_shape = (160, 160, 3)
        self.min_value = 0
        self.max_value = 10
        self.batch_size = 16
        self.checkpoint_dir = 'training_checkpoints'
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        
    def build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1)
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        print("Loading dataset...")
        dataset = load_dataset(
            "Synanthropic/reading-analog-gauge",
            split='train'
        )
        
        print(f"Total examples in dataset: {len(dataset)}")
        
        def preprocess_example(example):
            # Load and resize image
            image = np.array(example['image'])
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            image = image.astype(np.float32) / 255.0
            label = float(example['label'])
            return image, label

        def generator():
            for example in dataset:
                image, label = preprocess_example(example)
                yield image, label

        tf_dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )

        tf_dataset = tf_dataset.shuffle(1000)
        tf_dataset = tf_dataset.batch(self.batch_size)
        tf_dataset = tf_dataset.prefetch(tf.data.AUTOTUNE)

        steps_per_epoch = len(dataset) // self.batch_size

        return tf_dataset, steps_per_epoch

    def train_model(self, epochs=10):
        # Build model
        model = self.build_model()
        
        print("Preparing dataset...")
        train_ds, steps_per_epoch = self.prepare_dataset()
        print(f"Dataset prepared successfully! Steps per epoch: {steps_per_epoch}")
        
        # Define checkpoint paths with .keras extension
        checkpoint_path = os.path.join(self.checkpoint_dir, "epoch_{epoch:02d}.keras")
        best_model_path = os.path.join(self.checkpoint_dir, "best_model.keras")
        
        # Train model
        print("Starting training...")
        history = model.fit(
            train_ds,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            callbacks=[
                # Save after every epoch
                tf.keras.callbacks.ModelCheckpoint(
                    filepath=checkpoint_path,
                    save_weights_only=False,
                    save_freq='epoch'
                ),
                # Save best model based on loss
                tf.keras.callbacks.ModelCheckpoint(
                    filepath=best_model_path,
                    save_best_only=True,
                    monitor='loss',
                    mode='min',
                    save_weights_only=False
                ),
                tf.keras.callbacks.EarlyStopping(
                    monitor='loss',
                    patience=3,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    factor=0.5,
                    patience=2
                ),
                # Memory cleanup callback
                tf.keras.callbacks.LambdaCallback(
                    on_epoch_end=lambda epoch, logs: gc.collect()
                )
            ]
        )
        
        return model, history

    def load_latest_checkpoint(self):
        """Load the latest checkpoint if it exists."""
        checkpoints = [d for d in os.listdir(self.checkpoint_dir) 
                      if d.startswith('epoch_') and d.endswith('.keras')]
        if not checkpoints:
            return None
            
        latest_checkpoint = max(checkpoints)
        checkpoint_path = os.path.join(self.checkpoint_dir, latest_checkpoint)
        print(f"Loading checkpoint: {checkpoint_path}")
        return tf.keras.models.load_model(checkpoint_path)

    def convert_to_tflite(self, model):
        print("Converting to TFLite format...")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            pickle.dump(tflite_model, f)

def main():
    # Initialize GaugeReader
    gauge_reader = GaugeReader()
    
    try:
        # Check for existing checkpoints
        existing_model = gauge_reader.load_latest_checkpoint()
        if existing_model:
            print("Resuming from previous checkpoint...")
            model = existing_model
        else:
            # Train model
            print("Training new model...")
            model, history = gauge_reader.train_model(epochs=10)
        
        # Convert to TFLite
        print("Converting to TFLite...")
        tflite_model = gauge_reader.convert_to_tflite(model)
        
        # Save model
        print("Saving model...")
        gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
        
        # Plot training history if available
        if 'history' in locals():
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.plot(history.history['loss'], label='Loss')
            plt.title('Model Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            
            plt.subplot(1, 2, 2)
            plt.plot(history.history['mae'], label='MAE')
            plt.title('Model MAE')
            plt.xlabel('Epoch')
            plt.ylabel('MAE')
            plt.legend()
            
            plt.tight_layout()
            plt.show()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        # Clean up memory in case of error
        gc.collect()

if __name__ == "__main__":
    main()

Loading checkpoint: training_checkpoints/epoch_04.keras


2024-12-30 18:28:27.455764: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-30 18:28:27.455783: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-30 18:28:27.455786: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-30 18:28:27.455821: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-30 18:28:27.455835: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Resuming from previous checkpoint...
Converting to TFLite...
Converting to TFLite format...
INFO:tensorflow:Assets written to: /var/folders/rt/0x35vmdd32ldd69_8j6wrvgc0000gn/T/tmp1s0ea92r/assets


INFO:tensorflow:Assets written to: /var/folders/rt/0x35vmdd32ldd69_8j6wrvgc0000gn/T/tmp1s0ea92r/assets


: 

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15670774299620544029
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 13422472795634595793
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2024-12-30 18:30:26.618328: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-30 18:30:26.618445: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-30 18:30:26.618497: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-30 18:30:26.618562: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-30 18:30:26.618591: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## optimized for RTX 3060 gpu

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt

class GaugeReader:
    def __init__(self):
        # Increased input shape and batch size for better GPU utilization
        self.input_shape = (224, 224, 3)  # Larger input size
        self.min_value = 0
        self.max_value = 10
        self.batch_size = 64  # Larger batch size for GPU
        
    def build_model(self):
        # Deeper model architecture utilizing more GPU memory
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            
            # First conv block
            tf.keras.layers.Conv2D(64, 3, activation=None, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3, activation=None, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(),
            
            # Second conv block
            tf.keras.layers.Conv2D(128, 3, activation=None, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(128, 3, activation=None, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(),
            
            # Third conv block
            tf.keras.layers.Conv2D(256, 3, activation=None, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(256, 3, activation=None, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(),
            
            # Fourth conv block
            tf.keras.layers.Conv2D(512, 3, activation=None, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(512, 3, activation=None, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            
            # Dense layers
            tf.keras.layers.Dense(1024, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1)
        ])
        
        # Use mixed precision for faster training
        mixed_precision.set_global_policy('mixed_float16')
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        optimizer = mixed_precision.LossScaleOptimizer(optimizer)
        
        model.compile(
            optimizer=optimizer,
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        print("Loading dataset...")
        dataset = load_dataset("Synanthropic/reading-analog-gauge")
        train_ds = dataset['train']
        
        # Calculate split sizes
        total_size = len(train_ds)
        val_size = int(0.1 * total_size)  # 10% validation split
        train_size = total_size - val_size
        
        # Process all examples with parallel processing
        def preprocess_example(example):
            # Load and resize image
            image = np.array(example['image'])
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            image = image.astype(np.float32) / 255.0
            return image, example['label']
        
        # Create datasets
        train_dataset = tf.data.Dataset.from_tensor_slices(
            [preprocess_example(ex) for ex in train_ds[:train_size]]
        )
        val_dataset = tf.data.Dataset.from_tensor_slices(
            [preprocess_example(ex) for ex in train_ds[train_size:]]
        )
        
        # Optimize for GPU performance
        train_dataset = (train_dataset
            .shuffle(5000)
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
            .cache()
        )
        
        val_dataset = (val_dataset
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
            .cache()
        )
        
        return train_dataset, val_dataset

    def train_model(self, epochs=20):  # Increased epochs
        # Set memory growth for GPU
        physical_devices = tf.config.list_physical_devices('GPU')
        if physical_devices:
            for device in physical_devices:
                tf.config.experimental.set_memory_growth(device, True)
        
        # Build model
        model = self.build_model()
        
        print("Preparing dataset...")
        train_ds, val_ds = self.prepare_dataset()
        print("Dataset prepared successfully!")
        
        # Train model
        print("Starting training...")
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,
                    patience=3,
                    min_lr=1e-6
                ),
                tf.keras.callbacks.ModelCheckpoint(
                    'best_model.h5',
                    monitor='val_loss',
                    save_best_only=True
                )
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            pickle.dump(tflite_model, f)

def main():
    # Initialize GaugeReader
    gauge_reader = GaugeReader()
    
    # Train model
    print("Training model...")
    model, history = gauge_reader.train_model(epochs=20)
    
    # Convert to TFLite
    print("Converting to TFLite...")
    tflite_model = gauge_reader.convert_to_tflite(model)
    
    # Save model
    print("Saving model...")
    gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
    
    # Plot training history
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Training MAE')
    plt.plot(history.history['val_mae'], label='Validation MAE')
    plt.title('Model MAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()